In [1]:
import pandas as pd

import numpy as np

# Read in excel file of 2016 data
file = '/Users/Laurasaur/Desktop/Springboard/Datasets/2016/Table_13_Hate_Crime_Incidents_per_Bias_Motivation_and_Quarter_by_State_and_Agency_2016.xls'

xl = pd.ExcelFile(file)

# We want the first and only sheet
df2 = xl.parse(0)

# remove the first 4 rows, which contain the title
df2_sliced = df2.loc[5:]

# rename the cols
df2_sliced.columns = ['State', 'Agency_Type', 'Agency_Name', 'Race', 'Religion', 'Sexual_Orientation', 'Disability', 'Gender', 'Gender_ID', 'Q1', 'Q2', 'Q3', 'Q4', 'Pop']

# copy df, and then forward fill the state and agency type cols
df2_filled = df2_sliced

df2_filled[['State', 'Agency_Type']] = df2_filled[['State', 'Agency_Type']].ffill()

# copy and replace the NAs in df2_totals Agency Name col with the word total, because the NAs are only in cols w/ total
df2_totals = df2_filled

df2_totals = df2_totals.fillna(value={'Agency_Name' : 'Total'})
   
# reset index, so it starts at 1 not 5 
df2_indexed = df2_totals.reset_index()

print(df2_indexed.head())

   index    State Agency_Type Agency_Name Race Religion Sexual_Orientation  \
0      5  Alabama       Total       Total   12        1                  1   
1      6  Alabama      Cities       Total   12        1                  1   
2      7  Alabama      Cities  Birmingham    7        1                  1   
3      8  Alabama      Cities    Florence    1        0                  0   
4      9  Alabama      Cities       Foley    1        0                  0   

  Disability Gender Gender_ID   Q1   Q2   Q3   Q4     Pop  
0          0      0         0  NaN  NaN  NaN  NaN     NaN  
1          0      0         0  NaN  NaN  NaN  NaN     NaN  
2          0      0         0    3    2    2    2  212549  
3          0      0         0    0    0    1    0   40160  
4          0      0         0    0    1    0    0   17639  


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [2]:
#### I noticed some of the agency names contained numbers left over from subscript on the original data. I want to remove those.

df2_indexed['Agency_Name'] = df2_indexed.Agency_Name.str.replace('2', '')

df2_indexed['Agency_Name'] = df2_indexed.Agency_Name.str.replace('1', '')

df2_indexed.head()

,index,State,Agency_Type,Agency_Name,Race,Religion,Sexual_Orientation,Disability,Gender,Gender_ID,Q1,Q2,Q3,Q4,Pop
0,5,Alabama,Total,Total,12,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN
1,6,Alabama,Cities,Total,12,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN
2,7,Alabama,Cities,Birmingham,7,1,1,0,0,0,3,2,2,2,212549
3,8,Alabama,Cities,Florence,1,0,0,0,0,0,0,0,1,0,40160
4,9,Alabama,Cities,Foley,1,0,0,0,0,0,0,1,0,0,17639


In [4]:
## We're going to get the state abbreviations in order to match some of the datasets later
## we then make a dictionary, with the state names as keys and the abbreviations as values

state_file = '/Users/Laurasaur/Desktop/Springboard/Datasets/states.csv'

states = pd.read_csv(state_file)

states_dict = pd.Series(states.Abbreviation.values, index=states.State).to_dict()

print(states_dict)

{'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


In [6]:
## Now we create a new col in the dataset, containing the state abbreviations:

df2_abb= df2_indexed.copy()

df2_abb['Abbreviation'] = df2_abb['State'].map(states_dict)

print(df2_abb.head())

   index    State Agency_Type Agency_Name Race Religion Sexual_Orientation  \
0      5  Alabama       Total       Total   12        1                  1   
1      6  Alabama      Cities       Total   12        1                  1   
2      7  Alabama      Cities  Birmingham    7        1                  1   
3      8  Alabama      Cities    Florence    1        0                  0   
4      9  Alabama      Cities       Foley    1        0                  0   

  Disability Gender Gender_ID   Q1   Q2   Q3   Q4     Pop Abbreviation  
0          0      0         0  NaN  NaN  NaN  NaN     NaN           AL  
1          0      0         0  NaN  NaN  NaN  NaN     NaN           AL  
2          0      0         0    3    2    2    2  212549           AL  
3          0      0         0    0    0    1    0   40160           AL  
4          0      0         0    0    1    0    0   17639           AL  


In [7]:
## Next, we create another col that contains a string of the city name and the state abbreviation

df2_abb['City_State'] = df2_abb['Agency_Name'] + ", "  + df2_abb['Abbreviation']

print(df2_abb.head())

   index    State Agency_Type Agency_Name Race Religion Sexual_Orientation  \
0      5  Alabama       Total       Total   12        1                  1   
1      6  Alabama      Cities       Total   12        1                  1   
2      7  Alabama      Cities  Birmingham    7        1                  1   
3      8  Alabama      Cities    Florence    1        0                  0   
4      9  Alabama      Cities       Foley    1        0                  0   

  Disability Gender Gender_ID   Q1   Q2   Q3   Q4     Pop Abbreviation  \
0          0      0         0  NaN  NaN  NaN  NaN     NaN           AL   
1          0      0         0  NaN  NaN  NaN  NaN     NaN           AL   
2          0      0         0    3    2    2    2  212549           AL   
3          0      0         0    0    0    1    0   40160           AL   
4          0      0         0    0    1    0    0   17639           AL   

       City_State  
0       Total, AL  
1       Total, AL  
2  Birmingham, AL  
3    F

In [8]:
#### Now we get rid of every entry that's not a city (b/c we only want to examine cities)
### And we get rid of all the total agencies, as we won't use that information for out analysis

df2_cities = df2_abb[df2_abb.Agency_Type == 'Cities']

df2_cities = df2_cities[df2_cities.Agency_Name != 'Total']

df2_cities.head()

,index,State,Agency_Type,Agency_Name,Race,Religion,Sexual_Orientation,Disability,Gender,Gender_ID,Q1,Q2,Q3,Q4,Pop,Abbreviation,City_State
2,7,Alabama,Cities,Birmingham,7,1,1,0,0,0,3,2,2,2,212549,AL,"Birmingham, AL"
3,8,Alabama,Cities,Florence,1,0,0,0,0,0,0,0,1,0,40160,AL,"Florence, AL"
4,9,Alabama,Cities,Foley,1,0,0,0,0,0,0,1,0,0,17639,AL,"Foley, AL"
5,10,Alabama,Cities,Hoover,3,0,0,0,0,0,1,0,1,1,85601,AL,"Hoover, AL"
8,13,Alaska,Cities,Anchorage,4,0,0,0,0,0,1,0,2,1,299097,AK,"Anchorage, AK"


In [10]:
# Now we want to see how many datapoints were removed by taking out the cities and totals

len(df2_abb) - len(df2_cities)

754

In [11]:
##### Now we read in the dataset with the information about the counties, which we will use to match the
##### Hate Crime dataset with the dataset containing political and demographic information

county_file = '/Users/Laurasaur/Desktop/Springboard/Datasets/uscitiesv1.4.csv'

counties = pd.read_csv(county_file)

counties.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
0,Prairie Ridge,Prairie Ridge,WA,Washington,53053,Pierce,47.1443,-122.1408,NaN,NaN,1349.8,polygon,False,America/Los_Angeles,98360 98391,1840037882
1,Edison,Edison,WA,Washington,53057,Skagit,48.5602,-122.4311,NaN,NaN,127.4,polygon,False,America/Los_Angeles,98232,1840017314
2,Packwood,Packwood,WA,Washington,53041,Lewis,46.6085,-121.6702,NaN,NaN,213.9,polygon,False,America/Los_Angeles,98361,1840025265
3,Wautauga Beach,Wautauga Beach,WA,Washington,53035,Kitsap,47.5862,-122.5482,NaN,NaN,261.7,point,False,America/Los_Angeles,98366,1840037725
4,Harper,Harper,WA,Washington,53035,Kitsap,47.5207,-122.5196,NaN,NaN,342.1,point,False,America/Los_Angeles,98366,1840037659


In [12]:
## Now I'll make a col containing the state, city, and county, and remove all duplicates
## so that we have only unique sets (b/c there are duplicates in dataset because it also includes unique log and lat)

counties['City_County'] = counties.state_id + ", " + counties.city + ", " + counties.county_name

counties.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id,City_County
0,Prairie Ridge,Prairie Ridge,WA,Washington,53053,Pierce,47.1443,-122.1408,NaN,NaN,1349.8,polygon,False,America/Los_Angeles,98360 98391,1840037882,"WA, Prairie Ridge, Pierce"
1,Edison,Edison,WA,Washington,53057,Skagit,48.5602,-122.4311,NaN,NaN,127.4,polygon,False,America/Los_Angeles,98232,1840017314,"WA, Edison, Skagit"
2,Packwood,Packwood,WA,Washington,53041,Lewis,46.6085,-121.6702,NaN,NaN,213.9,polygon,False,America/Los_Angeles,98361,1840025265,"WA, Packwood, Lewis"
3,Wautauga Beach,Wautauga Beach,WA,Washington,53035,Kitsap,47.5862,-122.5482,NaN,NaN,261.7,point,False,America/Los_Angeles,98366,1840037725,"WA, Wautauga Beach, Kitsap"
4,Harper,Harper,WA,Washington,53035,Kitsap,47.5207,-122.5196,NaN,NaN,342.1,point,False,America/Los_Angeles,98366,1840037659,"WA, Harper, Kitsap"


In [13]:
### Now we create the seperate city, state, and county cols, and get rid of all the other info we don't need

counties3 = counties['City_County'].str.split(pat=', ', expand = True)

counties3.columns = ['State', 'City', 'County']

counties3['City_State'] = counties3.City + ", " + counties3.State

counties3.head()

,State,City,County,City_State
0,WA,Prairie Ridge,Pierce,"Prairie Ridge, WA"
1,WA,Edison,Skagit,"Edison, WA"
2,WA,Packwood,Lewis,"Packwood, WA"
3,WA,Wautauga Beach,Kitsap,"Wautauga Beach, WA"
4,WA,Harper,Kitsap,"Harper, WA"


In [1]:
#### Now we're going to make a dictionary, to match the city_state col with the county col

counties_dict1 = pd.Series(counties3.County.values, index=counties3.City_State).to_dict()

#print(counties_dict1)

NameError: name 'pd' is not defined

In [15]:
#### So now we have a dataset with all the cities and counties, which should help us determine the county for each city in our original dataframe

df2_counties = df2_cities.copy()

df2_counties['County'] = df2_counties['City_State'].map(counties_dict1)

print(df2_counties.head())

   index    State Agency_Type Agency_Name Race Religion Sexual_Orientation  \
2      7  Alabama      Cities  Birmingham    7        1                  1   
3      8  Alabama      Cities    Florence    1        0                  0   
4      9  Alabama      Cities       Foley    1        0                  0   
5     10  Alabama      Cities      Hoover    3        0                  0   
8     13   Alaska      Cities   Anchorage    4        0                  0   

  Disability Gender Gender_ID Q1 Q2 Q3 Q4     Pop Abbreviation  \
2          0      0         0  3  2  2  2  212549           AL   
3          0      0         0  0  0  1  0   40160           AL   
4          0      0         0  0  1  0  0   17639           AL   
5          0      0         0  1  0  1  1   85601           AL   
8          0      0         0  1  0  2  1  299097           AK   

       City_State      County  
2  Birmingham, AL   Jefferson  
3    Florence, AL  Lauderdale  
4       Foley, AL     Baldwin  
5     

In [18]:
### Now we want to know how many matches we can find, and thus how much information we can match

len(df2_counties)

1265

In [19]:
df2_counties['County'].isnull().sum()

196

In [20]:
(1265 - 196)/1265 # so we have the county info for about 85% of the cities, or 1069

0.8450592885375494

In [21]:
############ Now we read in the datasets that contain the election results
# clean that, rehape so each candidate is a col, and organize it by county

prez_file = '/Users/Laurasaur/Desktop/Springboard/Datasets/pres16results.csv'

prez_df = pd.read_csv(prez_file)

print(prez_df.head())

  county fips             cand  st  pct_report       votes  total_votes  \
0    NaN   US     Donald Trump  US      0.9951  60350241.0  127592176.0   
1    NaN   US  Hillary Clinton  US      0.9951  60981118.0  127592176.0   
2    NaN   US     Gary Johnson  US      0.9951   4164589.0  127592176.0   
3    NaN   US       Jill Stein  US      0.9951   1255968.0  127592176.0   
4    NaN   US    Evan McMullin  US      0.9951    451636.0  127592176.0   

        pct          lead  
0  0.472993  Donald Trump  
1  0.477938  Donald Trump  
2  0.032640  Donald Trump  
3  0.009844  Donald Trump  
4  0.003540  Donald Trump  


In [22]:
# removing the cols we're not going to use, and dropping ann NAs (because they indicate a total count, not per county)

prez_df1 = prez_df[['county', 'cand', 'st', 'pct']]

prez_df2 = prez_df1.dropna()

print(prez_df2.head())

                 county             cand  st       pct
159  Los Angeles County  Hillary Clinton  CA  0.714965
160  Los Angeles County     Donald Trump  CA  0.234454
161  Los Angeles County     Gary Johnson  CA  0.024589
162  Los Angeles County       Jill Stein  CA  0.020171
163  Los Angeles County   Gloria La Riva  CA  0.005821


In [23]:
### Now we remove the word county from the counties, so they can be matched

prez_df2['county'] = prez_df2.county.str.replace(' County', '')

print(prez_df2.head())

          county             cand  st       pct
159  Los Angeles  Hillary Clinton  CA  0.714965
160  Los Angeles     Donald Trump  CA  0.234454
161  Los Angeles     Gary Johnson  CA  0.024589
162  Los Angeles       Jill Stein  CA  0.020171
163  Los Angeles   Gloria La Riva  CA  0.005821


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
